In [1]:
import re
import string
import pymorphy2
import pandas as pd

In [2]:
aggregated = pd.read_csv('../data/marks_csv/aggregated.csv', index_col=None)

In [3]:
regex_ru = re.compile('[^а-яА-Я]')

def get_ru_words(line: str):
    words = [word for word in regex_ru.sub(' ', line).lower().split() if word]
    return words
aggregated['ru_words'] = aggregated.input.apply(get_ru_words)
aggregated.head(2)

,input,output,confidence,ru_words
0,Условия термодинамической согласованности имею...,NEUTRAL,0.92,"[условия, термодинамической, согласованности, ..."
1,"мишени, подтвержденная там для 2D  модели, до...",NEUTRAL,0.90,"[мишени, подтвержденная, там, для, модели, доп..."


In [4]:
from gensim.models.phrases import Phrases

DELIMITER = '_'
B_DELIMITER = DELIMITER.encode()

def train_phrases(sentences, n, min_count=30, threshold=50):
    if n < 2:
        raise ValueError("n < 2")
    
    grams = []
    transformed_corpus = []
    
    _words = sentences
    for ind in range(n - 1):
        gram = Phrases(_words, min_count=min_count, delimiter=B_DELIMITER, threshold=threshold)
        grams.append(gram)
        _words = gram[_words]
        transformed_corpus.append(_words)
            
    return grams, transformed_corpus

In [5]:
grams, transformed_corpus = train_phrases(aggregated.ru_words, 5, threshold=25)

In [6]:
phrases_by_groups = {name: list() for name in aggregated.output.unique()}
for index, (words, output) in enumerate(zip(transformed_corpus[-1], aggregated.output)):
    phrases = [word for word in words if DELIMITER in word]
    if phrases:
        phrases_by_groups[output].extend(phrases)

In [7]:
from collections import Counter
counts = {name: Counter(values) for name, values in phrases_by_groups.items()}

In [8]:
counts.keys()

dict_keys(['NEUTRAL', 'GOOD', 'BAD'])

In [12]:
counts['GOOD'].most_common()

[('на_основе', 189),
 ('а_также', 169),
 ('с_помощью', 150),
 ('при_этом', 124),
 ('может_быть', 113),
 ('не_только', 109),
 ('таким_образом', 107),
 ('настоящее_время', 72),
 ('данной_работе', 69),
 ('том_числе', 69),
 ('могут_быть', 67),
 ('с_использованием', 62),
 ('т_е', 62),
 ('за_счет', 58),
 ('с_учетом', 56),
 ('по_сравнению_с', 51),
 ('кроме_того', 51),
 ('точки_зрения', 47),
 ('см_например', 46),
 ('этом_случае', 44),
 ('одним_из', 44),
 ('одной_из', 43),
 ('том_что', 39),
 ('настоящей_работе', 37),
 ('один_из', 34),
 ('отметим_что', 34),
 ('из_за', 33),
 ('то_есть', 33),
 ('магнитного_поля', 33),
 ('зависимости_от', 32),
 ('соответствии_с', 32),
 ('т_д', 31),
 ('показано_что', 29),
 ('последнее_время', 29),
 ('при_помощи', 29),
 ('приводит_к', 28),
 ('как_правило', 27),
 ('представляет_собой', 27),
 ('зависит_от', 27),
 ('последние_годы', 27),
 ('применительно_к', 26),
 ('млрд_долл', 26),
 ('угловой_скорости', 25),
 ('протопланетного_диска', 25),
 ('тем_что', 24),
 ('тем_не_м

In [10]:
counts['NEUTRAL'].most_common()

[('на_основе', 312),
 ('а_также', 269),
 ('с_помощью', 250),
 ('при_этом', 191),
 ('т_е', 145),
 ('может_быть', 144),
 ('с_учетом', 115),
 ('см_например', 114),
 ('таким_образом', 112),
 ('с_использованием', 106),
 ('том_числе', 101),
 ('данной_работе', 92),
 ('не_только', 90),
 ('могут_быть', 88),
 ('за_счет', 83),
 ('зависимости_от', 82),
 ('настоящее_время', 80),
 ('настоящей_работе', 74),
 ('точки_зрения', 71),
 ('кроме_того', 69),
 ('приводит_к', 69),
 ('представляет_собой', 68),
 ('магнитного_поля', 67),
 ('этом_случае', 64),
 ('млрд_руб', 64),
 ('по_сравнению_с', 63),
 ('т_д', 61),
 ('зависит_от', 61),
 ('соответствии_с', 60),
 ('как_правило', 59),
 ('одним_из', 56),
 ('том_что', 56),
 ('российской_федерации', 54),
 ('численного_моделирования', 52),
 ('из_них', 52),
 ('по_данным', 52),
 ('один_из', 51),
 ('из_за', 48),
 ('млрд_долл', 48),
 ('протопланетного_диска', 47),
 ('то_есть', 45),
 ('дифференциальных_уравнений', 45),
 ('одной_из', 44),
 ('предполагается_что', 44),
 ('цб_р

In [11]:
counts['BAD'].most_common()

[('при_этом', 117),
 ('на_основе', 98),
 ('может_быть', 86),
 ('из_за', 84),
 ('а_также', 81),
 ('с_помощью', 70),
 ('таким_образом', 70),
 ('не_только', 63),
 ('т_е', 60),
 ('по_сравнению_с', 60),
 ('кроме_того', 49),
 ('точки_зрения', 48),
 ('приводит_к', 46),
 ('к_сожалению', 46),
 ('за_счет', 45),
 ('том_числе', 44),
 ('настоящее_время', 42),
 ('несмотря_на', 42),
 ('том_что', 38),
 ('данной_работе', 37),
 ('тем_не_менее', 37),
 ('как_правило', 37),
 ('по_мнению', 36),
 ('цб_рф', 36),
 ('могут_быть', 35),
 ('российской_федерации', 34),
 ('вопрос_о', 33),
 ('то_есть', 33),
 ('см_например', 32),
 ('отличие_от', 30),
 ('тем_что', 29),
 ('то_же_время', 28),
 ('настоящей_работе', 28),
 ('по_данным', 28),
 ('того_что', 28),
 ('млрд_долл', 28),
 ('при_использовании', 27),
 ('из_них', 27),
 ('то_время_как', 27),
 ('с_учетом', 26),
 ('о_том_что', 26),
 ('российской_экономики', 26),
 ('зависимости_от', 25),
 ('млрд_руб', 24),
 ('соответствии_с', 24),
 ('тогда_как', 24),
 ('т_д', 24),
 ('было